<a href="https://colab.research.google.com/github/R-aryan/Image_Classification_VGG16/blob/master/Classification_Cat_VS_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!unzip '/content/drive/My Drive/classification_Dataset/cat_VS_dogs/test1.zip' -d '/content/drive/My Drive/classification_Dataset/cat_VS_dogs'

In [13]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.


In [0]:
train_directory= "/content/drive/My Drive/classification_Dataset/cat_VS_dogs/train"
test_directory="/content/drive/My Drive/classification_Dataset/cat_VS_dogs/test1"

In [0]:
import shutil
import os
from os import listdir
from os.path import splitext

In [0]:
src= '/content/drive/My Drive/classification_Dataset/cat_VS_dogs/train'
dest_d='/content/drive/My Drive/classification_Dataset/cat_VS_dogs/train/Dogs'
dest_c='/content/drive/My Drive/classification_Dataset/cat_VS_dogs/train/Cats'
validation_set='/content/drive/My Drive/classification_Dataset/cat_VS_dogs/validation_data'

In [0]:
for i in range (20):
    random_index = random.randint(1,992)
#     print(random_index)
    try:
        src = os.path.join(source,str(random_index)+'.jpg')
        shutil.move(src, dest)
    except:
        print('File Already Exist '+str(random_index)+'.jpg')
print('Done')

In [15]:
c=1
for file in listdir(dest_d):
    filename = file[:3]
    #f1=filename.lower()
    # print(filename)
    c+=1
    
    # try:
    #   if(filename=='cat'):
    #     shutil.move(os.path.join(src,file), dest_c)
    #     print("cat moved succesfully")
    #   else:
    #     shutil.move(os.path.join(src,file),dest_d)
    #     print("dog moved successfully")

    # except:
    #   print("error occured at "+ file)

    # if(c==10):
    #   break

print(c)

7997


In [16]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory=src,target_size=(224,224))

Found 20438 images belonging to 2 classes.


In [18]:
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory=validation_set, target_size=(224,224))

Found 98 images belonging to 2 classes.
